In [100]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import yfinance as yf

In [101]:
def clean(ticker):
    ydf = yf.download(ticker, interval="1d")
    #ydf.to_csv(f'y{ticker}.csv')
    ydf.to_csv(f'test.csv')

    # btc = pd.read_csv(f'y{ticker}.csv')
    btc = pd.read_csv(f'test.csv')
    btc = btc.drop([0,1])
    btc['Date'] = pd.to_datetime(btc['Price']).dt.date
    # btc['Date'] = pd.to_datetime(btc['Price']).dt.floor('T')
    btc= btc.set_index('Date')
    btc = btc.drop(['Price'], axis = 1)
    btc = btc.reset_index()
    #btc['Ticker'] = ticker
    return btc

In [102]:
btc = clean('BTC-USD')
#btc['ret'] = btc['Adj Close'].pct_change()

btc[btc.columns[1:]] = btc[btc.columns[1:]].astype(float)

btc

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2014-09-17,457.334015,457.334015,468.174011,452.421997,465.864014,2.105680e+07
1,2014-09-18,424.440002,424.440002,456.859985,413.104004,456.859985,3.448320e+07
2,2014-09-19,394.795990,394.795990,427.834991,384.532013,424.102997,3.791970e+07
3,2014-09-20,408.903992,408.903992,423.295990,389.882996,394.673004,3.686360e+07
4,2014-09-21,398.821014,398.821014,412.425995,393.181000,408.084991,2.658010e+07
...,...,...,...,...,...,...,...
3703,2024-11-06,75639.078125,75639.078125,76460.156250,69322.031250,69358.500000,1.185927e+11
3704,2024-11-07,75904.859375,75904.859375,76943.117188,74480.421875,75637.085938,6.346765e+10
3705,2024-11-08,76545.476562,76545.476562,77252.750000,75648.742188,75902.835938,5.517686e+10
3706,2024-11-09,76778.867188,76778.867188,76932.765625,75773.789062,76556.187500,2.900948e+10


In [103]:
btc['Date'].min()

datetime.date(2014, 9, 17)

In [104]:
btc['ret'] = btc['Adj Close'].pct_change()
btc['logret'] = np.log1p(btc['ret'])

vol_windows = [7, 15, 30, 60]
#do we need to annualize? prob not
for i in vol_windows:
    btc[f'vol_{i}'] = btc['logret'].rolling(window=i).std()


btc['Gain'] = btc['logret'].apply(lambda x: x if x > 0 else 0)
btc['Loss'] = btc['logret'].apply(lambda x: -x if x < 0 else 0)


look_backs = [15, 30, 60, 120, 240]
for l in look_backs:
    # btc[f'Avg_Gain_{l}'] = btc['Gain'].rolling(window=l).mean()
    # btc[f'Avg_Loss_{l}'] = btc['Loss'].rolling(window=l).mean()
    # btc[f'RSI_{l}'] = 100 - (100 / (1 + (btc[f'Avg_Gain_{l}'] / btc[f'Avg_Loss_{l}'])))
    
    btc[f'RSI_{l}'] = 100 - (100 / (1 + (btc['Gain'].rolling(window=l).mean() / btc['Loss'].rolling(window=l).mean())))

btc

mom_windows = [7, 30, 60, 180, 365]
for m in mom_windows:
    btc[f'mom_{m}'] = (1/m) * btc['logret'].rolling(m,m).sum().reset_index(drop=True)

btc['Return'].ewm(span=20, adjust=False).mean()
btc


,Date,Adj Close,Close,High,Low,Open,Volume,ret,logret,vol_5,vol_10,vol_15,vol_20,Gain,Loss,RSI_15,RSI_30,RSI_60,RSI_120,RSI_240
0,2014-09-17,457.334015,457.334015,468.174011,452.421997,465.864014,2.105680e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
1,2014-09-18,424.440002,424.440002,456.859985,413.104004,456.859985,3.448320e+07,-0.071926,-0.074643,NaN,NaN,NaN,NaN,0.000000,0.074643,NaN,NaN,NaN,NaN,NaN
2,2014-09-19,394.795990,394.795990,427.834991,384.532013,424.102997,3.791970e+07,-0.069843,-0.072402,NaN,NaN,NaN,NaN,0.000000,0.072402,NaN,NaN,NaN,NaN,NaN
3,2014-09-20,408.903992,408.903992,423.295990,389.882996,394.673004,3.686360e+07,0.035735,0.035111,NaN,NaN,NaN,NaN,0.035111,0.000000,NaN,NaN,NaN,NaN,NaN
4,2014-09-21,398.821014,398.821014,412.425995,393.181000,408.084991,2.658010e+07,-0.024659,-0.024968,NaN,NaN,NaN,NaN,0.000000,0.024968,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3703,2024-11-06,75639.078125,75639.078125,76460.156250,69322.031250,69358.500000,1.185927e+11,0.090536,0.086669,0.041339,0.034202,0.029717,0.026637,0.086669,0.000000,67.639381,68.104051,66.719592,55.699045,50.488666
3704,2024-11-07,75904.859375,75904.859375,76943.117188,74480.421875,75637.085938,6.346765e+10,0.003514,0.003508,0.040678,0.033655,0.029147,0.026551,0.003508,0.000000,70.949343,68.523210,66.404707,55.876128,50.617279
3705,2024-11-08,76545.476562,76545.476562,77252.750000,75648.742188,75902.835938,5.517686e+10,0.008440,0.008404,0.038654,0.031839,0.028775,0.026521,0.008404,0.000000,69.280102,72.334354,65.336056,56.201170,50.477146
3706,2024-11-09,76778.867188,76778.867188,76932.765625,75773.789062,76556.187500,2.900948e+10,0.003049,0.003044,0.035455,0.031646,0.027586,0.026512,0.003044,0.000000,75.160429,73.201826,65.047388,56.077063,50.752434


In [ ]:
df = pd.read_csv("proj_data.csv", index_col = 0)
df = df[df['tick']=='BTC-USD']
df['ret'] = df['adj_prc'].pct_change()
df

,date,adj_prc,prc,high,low,open,vol,tick,ret,logret,mom,disp
3702,2024-11-05,69359.562500,69359.562500,70522.789062,67458.867188,67811.171875,4.604689e+10,BTC-USD,NaN,0.022577,0.386823,0.045183
3701,2024-11-04,67811.507812,67811.507812,69433.179688,66803.648438,68742.132812,4.118482e+10,BTC-USD,-0.022319,-0.013630,0.431593,0.038252
3700,2024-11-03,68741.117188,68741.117188,69361.656250,67482.523438,69296.382812,3.486831e+10,BTC-USD,0.013709,-0.008045,0.419293,0.027117
3699,2024-11-02,69289.273438,69289.273438,69867.351562,69033.718750,69486.023438,1.818461e+10,BTC-USD,0.007974,-0.002836,0.405636,0.011997
3698,2024-11-01,69482.468750,69482.468750,71559.015625,68779.703125,70216.898438,4.998980e+10,BTC-USD,0.002788,-0.010515,0.376128,0.039582
...,...,...,...,...,...,...,...,...,...,...,...,...
1486,2018-10-12,6274.580078,6274.580078,6328.500000,6236.470215,6239.250000,3.783500e+09,BTC-USD,-0.001815,0.005647,-0.013676,0.014750
1485,2018-10-11,6256.240234,6256.240234,6586.740234,6243.740234,6586.740234,5.181640e+09,BTC-USD,-0.002923,-0.051479,-0.013555,0.052074
1484,2018-10-10,6585.529785,6585.529785,6640.290039,6538.959961,6640.290039,3.787650e+09,BTC-USD,0.052634,-0.008281,-0.095089,0.015260
1483,2018-10-09,6642.640137,6642.640137,6661.410156,6606.939941,6653.080078,3.580810e+09,BTC-USD,0.008672,-0.001570,-0.108558,0.008187


In [ ]:
df['date'].min()

'2018-10-08'

In [ ]:
df['ret_2'] = df['adj_prc'].pct_change()
df2 = df[['ret', 'ret_2']]
df2

,ret,ret_2
3702,NaN,NaN
3701,-0.022319,-0.022319
3700,0.013709,0.013709
3699,0.007974,0.007974
3698,0.002788,0.002788
...,...,...
1486,-0.001815,-0.001815
1485,-0.002923,-0.002923
1484,0.052634,0.052634
1483,0.008672,0.008672
